In [1]:
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import spectral.io.envi as envi
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn import preprocessing
from scipy.signal import savgol_filter
from scipy.ndimage import gaussian_filter
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from pathlib import Path
from PIL import Image
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
import os,glob
import h5py
from skimage.transform import resize
from skimage.transform import rotate


In [2]:
datafolderpath="/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/"

In [3]:
#load Path Data
def loadPatientData():
    patientdata=[]

    # Load Patient Data
    for filename in os.listdir(datafolderpath):
        subdir= os.path.join(datafolderpath, filename)
        if not os.path.isfile(subdir):
            patientfiledata={}
            rotationfiles=[]
            maskfiles=[]
            patientfiledata['pat_no']=filename
            patientdata.append(patientfiledata)

            for subf in glob.glob(subdir+"/spec_256_rot*"):
                rotationfiles.append(subf)
            for subf in glob.glob(subdir+"/mask_256_rot*"):
                maskfiles.append(subf)

            patientfiledata['spec_data']=subdir+"/spec_256.npy"
            patientfiledata['mask_data']=subdir+"/mask_256.npy"
            patientfiledata['rotatiofiles']=rotationfiles
            patientfiledata['maskfiles']=maskfiles
            break


    print(patientdata)
    return patientdata
data=loadPatientData()

[{'pat_no': '004-02', 'spec_data': '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256.npy', 'mask_data': '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/mask_256.npy', 'rotatiofiles': ['/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256_rot_30.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256_rot_45.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256_rot_60.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256_rot_75.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/spec_256_rot_90.npy'], 'maskfiles': ['/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/mask_256_rot_30.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/mask_256_rot_45.npy', '/../../../data/mudahmad/HSI_Human_Brain_Database_IEEE_Access/004-02/mask_256_rot_60.npy', '/../../../data/mudahmad

In [4]:
#load File Data
def loadData(path):
     with open(path, 'rb') as f:
        arr = np.load(f,allow_pickle=True)
        return arr

In [10]:
# f, axs = plt.subplots(1,2,figsize=(15,15))
pat=data[0]
spec_data=loadData(pat['spec_data'])
mask_data=loadData(pat['mask_data'])

print(spec_data.shape)
print(mask_data.shape)


print(mask_data.reshape(256*256,)[0])

label_data=tf.one_hot(
    mask_data,#your image with label
    5, #the number of classes
    on_value=None,
    off_value=None,
    axis=None,
    dtype=None,
    name=None
)
print(label_data)

(256, 256, 700)
(256, 256, 1)
0
tf.Tensor(
[[[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]


 ...


 [[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]


 [[[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  ...

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]

  [[1. 0. 0. 0. 0.]]]], shape=(256, 256, 1, 5), dtype=float32)


In [13]:
mask_data.reshape(256,256)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])